In [1]:
import pandas as pd
import requests
import re
import lxml
from bs4 import BeautifulSoup

### User Agent 를 이용하여 200 코드 받기

-----------------------------

브라우저가 많아지고 호환성의 문제가 발생하기 때문에 호환성이 대표적으로 좋은편인 **'Firefox'**, **'Chrome'** 중 Firefox를 선택

In [2]:
chart_url = 'https://music.bugs.co.kr/chart'

headers = {
    'User_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:10.0) Gecko/20100101 Firefox/10.0'
}

In [3]:
res = requests.get(chart_url, headers=headers)
res

<Response [200]>

In [4]:
html = res.text

In [5]:
soup = BeautifulSoup(html, 'html.parser')

### pandas

----------------------

**pandas** 의 자료구조인 `data_frame`과 `series` 로 크롤링 해보기

In [6]:
res = requests.get(chart_url, headers)
html = res.text

In [7]:
type(pd.read_html(html))

list

In [8]:
len(pd.read_html(html))

3

In [9]:
chart_df = pd.read_html(html)[0]
chart_df

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타
0,NaN,1 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
1,NaN,2 1 계단 상승,NaN,곡정보,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
2,NaN,3 1 계단 하락,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
3,NaN,4 0변동없음,NaN,곡정보,고백,양다일,고백,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
4,NaN,5 1 계단 상승,NaN,곡정보,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition],듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
5,NaN,6 1 계단 하락,NaN,곡정보,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
6,NaN,7 0변동없음,NaN,곡정보,시간이 들겠지 (Feat. Colde),로꼬,시간이 들겠지,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
7,NaN,8 0변동없음,NaN,곡정보,가을 타나 봐,바이브(Vibe),가을 타나 봐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
8,NaN,9 0변동없음,NaN,곡정보,하루도 그대를 사랑하지 않은 적이 없었다,임창정,하루도 그대를 사랑하지 않은 적이 없었다,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
9,NaN,10 2 계단 상승,NaN,곡정보,물어,나플라(nafla),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능


In [10]:
chart_df.columns

Index(['Unnamed: 0', '순위', 'Unnamed: 2', 'Unnamed: 3', '곡', '아티스트', '앨범', '듣기',
       '재생목록', '내앨범', '다운', '영상', '기타'],
      dtype='object')

In [11]:
chart_df['곡']

0                                        삐삐
1           Save (Feat. 팔로알토)(Prod. 코드 쿤스트)
2       멋지게 인사하는 법 (Feat. 슬기 of Red Velvet)
3                                        고백
4                          Kiss and Make Up
5      Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트)
6                     시간이 들겠지 (Feat. Colde)
7                                   가을 타나 봐
8                    하루도 그대를 사랑하지 않은 적이 없었다
9                                        물어
10                                   어찌 잊으오
11                                 넌 (none)
12                                   꿈처럼 내린
13                      브레이킹배드 (Feat. 기리보이)
14                            Way Back Home
15                               사이렌(Siren)
16                       이별길 (GOODBYE ROAD)
17                   XXL (Feat. 딥플로우, Dok2)
18    모든 날, 모든 순간 (Every day, Every Moment)
19                    빌어먹을 인연 (Feat. Sik-K)
20      Hate You (Feat. 우원재) (Prod. 코드 쿤스트)
21                                  우리 그만하자
22            Balance (Prod. By 

In [12]:
names = ['곡', '아티스트']
chart_df[names]

,곡,아티스트
0,삐삐,아이유(IU)
1,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy)
2,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T
3,고백,양다일
4,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파)
5,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1
6,시간이 들겠지 (Feat. Colde),로꼬
7,가을 타나 봐,바이브(Vibe)
8,하루도 그대를 사랑하지 않은 적이 없었다,임창정
9,물어,나플라(nafla)
